# indicators

In [1]:
import vectorbt as vbt

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from numba import njit
import itertools
import talib
import ta

In [3]:
# Disable caching for performance testing
vbt.settings.caching['enabled'] = False

In [4]:
close = pd.DataFrame({
    'a': [1., 2., 3., 4., 5.],
    'b': [5., 4., 3., 2., 1.],
    'c': [1., 2., 3., 2., 1.]
}, index=pd.DatetimeIndex([
    datetime(2018, 1, 1),
    datetime(2018, 1, 2),
    datetime(2018, 1, 3),
    datetime(2018, 1, 4),
    datetime(2018, 1, 5)
]))
np.random.seed(42)
high = close * np.random.uniform(1, 1.1, size=close.shape)
low = close * np.random.uniform(0.9, 1, size=close.shape)
volume = close * 0 + np.random.randint(1, 10, size=close.shape).astype(float)

In [5]:
big_close = pd.DataFrame(np.random.randint(10, size=(1000, 1000)).astype(float))
big_close.index = [datetime(2018, 1, 1) + timedelta(days=i) for i in range(1000)]
big_high = big_close * np.random.uniform(1, 1.1, size=big_close.shape)
big_low = big_close * np.random.uniform(0.9, 1, size=big_close.shape)
big_volume = big_close * 0 + np.random.randint(10, 100, size=big_close.shape).astype(float)

In [6]:
close_ts = pd.Series([1, 2, 3, 4, 3, 2, 1], index=pd.DatetimeIndex([
    datetime(2018, 1, 1),
    datetime(2018, 1, 2),
    datetime(2018, 1, 3),
    datetime(2018, 1, 4),
    datetime(2018, 1, 5),
    datetime(2018, 1, 6),
    datetime(2018, 1, 7)
]))
high_ts = close_ts * 1.1
low_ts = close_ts * 0.9
volume_ts = pd.Series([4, 3, 2, 1, 2, 3, 4], index=close_ts.index)

## IndicatorFactory

In [7]:
def apply_func(i, ts, p, a, b=100):
    return ts * p[i] + a + b

@njit
def apply_func_nb(i, ts, p, a, b):
    return ts * p[i] + a + b # numba doesn't support **kwargs

# Custom function can be anything that takes time series, params and other arguments, and returns outputs
def custom_func(ts, p, *args, **kwargs):
    return vbt.base.combine_fns.apply_and_concat_one(len(p), apply_func, ts, p, *args, **kwargs)

@njit
def custom_func_nb(ts, p, *args):
    return vbt.base.combine_fns.apply_and_concat_one_nb(len(p), apply_func_nb, ts, p, *args)

F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p'], output_names=['out'])
print(F.from_custom_func(custom_func, var_args=True)
      .run(close, [0, 1], 10, b=100).out)
print(F.from_custom_func(custom_func_nb, var_args=True)
      .run(close, [0, 1], 10, 100).out)

In [8]:
# Apply function is performed on each parameter individually, and each output is then stacked for you
# Apply functions are less customizable than custom functions, but are simpler to write
def apply_func(ts, p, a, b=100):
    return ts * p + a + b

@njit
def apply_func_nb(ts, p, a, b):
    return ts * p + a + b  # numba doesn't support **kwargs
     
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p'], output_names=['out'])
print(F.from_apply_func(apply_func, var_args=True)
      .run(close, [0, 1], 10, b=100).out)
print(F.from_apply_func(apply_func_nb, var_args=True)
      .run(close, [0, 1], 10, 100).out)

In [9]:
# test *args
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p'], output_names=['out'])
print(F.from_apply_func(lambda ts, p, a: ts * p + a, var_args=True)
      .run(close, [0, 1, 2], 3).out) 
print(F.from_apply_func(njit(lambda ts, p, a: ts * p + a), var_args=True)
      .run(close, [0, 1, 2], 3).out)

In [10]:
# test **kwargs
# Numba doesn't support kwargs out of the box
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p'], output_names=['out'])
print(F.from_apply_func(lambda ts, p, a=1: ts * p + a)
      .run(close, [0, 1, 2], a=3).out) 

In [11]:
# test no inputs
F = vbt.IndicatorFactory(param_names=['p'], output_names=['out'])
print(F.from_apply_func(lambda p: np.full((3, 3), p))
      .run([0, 1]).out)
print(F.from_apply_func(njit(lambda p: np.full((3, 3), p)))
      .run([0, 1]).out)

In [12]:
# test no inputs with input_shape, input_index and input_columns
F = vbt.IndicatorFactory(param_names=['p'], output_names=['out'])
print(F.from_apply_func(lambda input_shape, p: np.full(input_shape, p), require_input_shape=True)
      .run((5,), 0).out)
print(F.from_apply_func(njit(lambda input_shape, p: np.full(input_shape, p)), require_input_shape=True)
      .run((5,), 0).out)

print(F.from_apply_func(lambda input_shape, p: np.full(input_shape, p), require_input_shape=True)
      .run((5,), [0, 1]).out)
print(F.from_apply_func(njit(lambda input_shape, p: np.full(input_shape, p)), require_input_shape=True)
      .run((5,), [0, 1]).out)

print(F.from_apply_func(lambda input_shape, p: np.full(input_shape, p), require_input_shape=True)
      .run((5, 3), [0, 1], input_index=close.index, input_columns=close.columns).out)
print(F.from_apply_func(njit(lambda input_shape, p: np.full(input_shape, p)), require_input_shape=True)
      .run((5, 3), [0, 1], input_index=close.index, input_columns=close.columns).out)

In [13]:
# test multiple inputs
F = vbt.IndicatorFactory(input_names=['ts1', 'ts2'], param_names=['p'], output_names=['out'])
print(F.from_apply_func(lambda ts1, ts2, p: ts1 * ts2 * p)
      .run(close, high, [0, 1]).out)
print(F.from_apply_func(njit(lambda ts1, ts2, p: ts1 * ts2 * p))
      .run(close, high, [0, 1]).out)

In [14]:
# test no params
F = vbt.IndicatorFactory(input_names=['ts'], output_names=['out'])
print(F.from_apply_func(lambda ts: ts)
      .run(close).out)
print(F.from_apply_func(njit(lambda ts: ts))
      .run(close).out)

In [15]:
# test no inputs and no params
F = vbt.IndicatorFactory(output_names=['out'])
print(F.from_apply_func(lambda: np.full((3, 3), 1))
      .run().out)
print(F.from_apply_func(njit(lambda: np.full((3, 3), 1)))
      .run().out)

In [16]:
# test multiple params
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p1', 'p2'], output_names=['out'])
print(F.from_apply_func(lambda ts, p1, p2: ts * (p1 + p2))
      .run(close, np.asarray([0, 1]), np.asarray([2, 3])).out) 
print(F.from_apply_func(njit(lambda ts, p1, p2: ts * (p1 + p2)))
      .run(close, np.asarray([0, 1]), np.asarray([2, 3])).out)

In [17]:
# test param_settings array_like
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p1', 'p2'], output_names=['out'])
print(F.from_apply_func(lambda ts, p1, p2: ts * (p1 + p2), 
                        param_settings={'p1': {'is_array_like': True}})
      .run(close, np.asarray([0, 1, 2]), np.asarray([2, 3])).out) 
print(F.from_apply_func(njit(lambda ts, p1, p2: ts * (p1 + p2)), 
                        param_settings={'p1': {'is_array_like': True}})
      .run(close, np.asarray([0, 1, 2]), np.asarray([2, 3])).out)

In [18]:
# test param_settings bc_to_input
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p1', 'p2'], output_names=['out'])
print(F.from_apply_func(lambda ts, p1, p2: ts * (p1 + p2), 
                        param_settings={'p1': {'is_array_like': True, 'bc_to_input': True}})
      .run(close, np.asarray([0, 1, 2]), np.asarray([2, 3])).out) 
print(F.from_apply_func(njit(lambda ts, p1, p2: ts * (p1 + p2)), 
                        param_settings={'p1': {'is_array_like': True, 'bc_to_input': True}})
      .run(close, np.asarray([0, 1, 2]), np.asarray([2, 3])).out)

In [19]:
# test param product
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p1', 'p2'], output_names=['out'])
print(F.from_apply_func(lambda ts, p1, p2: ts * (p1 + p2))
      .run(close, [0, 1], [2, 3], param_product=True).out) 
print(F.from_apply_func(njit(lambda ts, p1, p2: ts * (p1 + p2)))
      .run(close, [0, 1], [2, 3], param_product=True).out)

In [20]:
# test default params
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p1', 'p2'], output_names=['out'])
print(F.from_apply_func(lambda ts, p1, p2: ts * (p1 + p2), p2=2)
      .run(close, [0, 1]).out)
print(F.from_apply_func(njit(lambda ts, p1, p2: ts * (p1 + p2)), p2=2)
      .run(close, [0, 1]).out)

In [21]:
# test hide_params
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p1', 'p2'], output_names=['out'])
print(F.from_apply_func(lambda ts, p1, p2: ts * (p1 + p2), hide_params=['p2'])
      .run(close, [0, 1], 2).out)
print(F.from_apply_func(njit(lambda ts, p1, p2: ts * (p1 + p2)), hide_params=['p2'])
      .run(close, [0, 1], 2).out)

In [22]:
# test hide_default
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p1', 'p2'], output_names=['out'])
print(F.from_apply_func(lambda ts, p1, p2: ts * (p1 + p2), p2=2)
      .run(close, [0, 1], hide_default=False).out)
print(F.from_apply_func(njit(lambda ts, p1, p2: ts * (p1 + p2)), p2=2)
      .run(close, [0, 1], hide_default=False).out)
print(F.from_apply_func(lambda ts, p1, p2: ts * (p1 + p2), p2=2)
      .run(close, [0, 1], hide_default=True).out)
print(F.from_apply_func(njit(lambda ts, p1, p2: ts * (p1 + p2)), p2=2)
      .run(close, [0, 1], hide_default=True).out)

In [23]:
# test multiple outputs
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p'], output_names=['o1', 'o2'])
print(F.from_apply_func(lambda ts, p: (ts * p, ts * p ** 2))
      .run(close, [0, 1]).o1)
print(F.from_apply_func(lambda ts, p: (ts * p, ts * p ** 2))
      .run(close, [0, 1]).o2)
print(F.from_apply_func(njit(lambda ts, p: (ts * p, ts * p ** 2)))
      .run(close, [0, 1]).o1)
print(F.from_apply_func(njit(lambda ts, p: (ts * p, ts * p ** 2)))
      .run(close, [0, 1]).o2)

In [24]:
# test in-place outputs
def apply_func(ts, ts_out, p):
    ts_out[:, 0] = p
    return ts * p

F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p'], output_names=['out'], in_output_names=['ts_out'])
print(F.from_apply_func(apply_func)
      .run(close, [0, 1]).ts_out)
print(F.from_apply_func(njit(apply_func))
      .run(close, [0, 1]).ts_out)

print(F.from_apply_func(apply_func, in_output_settings={'ts_out': {'dtype': np.int_}})
      .run(close, [0, 1]).ts_out)
print(F.from_apply_func(njit(apply_func), in_output_settings={'ts_out': {'dtype': np.int_}})
      .run(close, [0, 1]).ts_out)

print(F.from_apply_func(apply_func, ts_out=-1)
      .run(close, [0, 1]).ts_out)
print(F.from_apply_func(njit(apply_func), ts_out=-1)
      .run(close, [0, 1]).ts_out)

In [25]:
# test kwargs_to_args
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p'], output_names=['out'])
print(F.from_apply_func(lambda ts, p, a, kw: ts * p + a + kw, kwargs_to_args=['kw'], var_args=True)
      .run(close, [0, 1, 2], 3, kw=10).out) 
print(F.from_apply_func(njit(lambda ts, p, a, kw: ts * p + a + kw), kwargs_to_args=['kw'], var_args=True)
      .run(close, [0, 1, 2], 3, kw=10).out)

In [26]:
# test caching func
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p'], output_names=['out'])
print(F.from_apply_func(lambda ts, param, c: ts * param + c, cache_func=lambda ts, params: 100)
      .run(close, [0, 1]).out)
print(F.from_apply_func(njit(lambda ts, param, c: ts * param + c), cache_func=njit(lambda ts, params: 100))
      .run(close, [0, 1]).out)

In [27]:
# test run_combs
F = vbt.IndicatorFactory(input_names=['ts'], param_names=['p1', 'p2'], output_names=['out'])
print(F.from_apply_func(lambda ts, p1, p2: ts * (p1 + p2))
      .run_combs(close, [0, 1, 2], [3, 4, 5], short_names=['i1', 'i2'])[0].out)
print(F.from_apply_func(lambda ts, p1, p2: ts * (p1 + p2))
      .run_combs(close, [0, 1, 2], [3, 4, 5], short_names=['i1', 'i2'])[1].out)
print(F.from_apply_func(njit(lambda ts, p1, p2: ts * (p1 + p2)))
      .run_combs(close, [0, 1, 2], [3, 4, 5], short_names=['i1', 'i2'])[0].out)
print(F.from_apply_func(njit(lambda ts, p1, p2: ts * (p1 + p2)))
      .run_combs(close, [0, 1, 2], [3, 4, 5], short_names=['i1', 'i2'])[1].out)

In [28]:
from collections import namedtuple

TestEnum = namedtuple('TestEnum', ['Hello', 'World'])(0, 1)
# test attr_settings
F = vbt.IndicatorFactory(
    input_names=['ts'], output_names=['o1', 'o2'], in_output_names=['ts_out'],
    attr_settings={
        'ts': {'dtype': None}, 
        'o1': {'dtype': np.float_}, 
        'o2': {'dtype': np.bool_}, 
        'ts_out': {'dtype': TestEnum}
    }
)
dir(F.from_apply_func(lambda ts, ts_out: (ts + ts_out, ts + ts_out)).run(close))

In [29]:
CustomInd = vbt.IndicatorFactory(
    input_names=['ts1', 'ts2'],
    param_names=['p1', 'p2'],
    output_names=['o1', 'o2']
).from_apply_func(lambda ts1, ts2, p1, p2: (ts1 * p1, ts2 * p2))

In [30]:
dir(CustomInd) # you can list here all of the available tools

In [31]:
custom_ind = CustomInd.run(close, high, [1, 2], [3, 4])
big_custom_ind = CustomInd.run(big_close, big_high, [1, 2], [3, 4])

In [32]:
print(custom_ind.wrapper.index) # subclasses ArrayWrapper
print(custom_ind.wrapper.columns)
print(custom_ind.wrapper.ndim)
print(custom_ind.wrapper.shape)
print(custom_ind.wrapper.freq)

In [33]:
# not changed during indexing
print(custom_ind.short_name)
print(custom_ind.level_names)
print(custom_ind.input_names)
print(custom_ind.param_names)
print(custom_ind.output_names)
print(custom_ind.output_flags)
print(custom_ind.p1_list)
print(custom_ind.p2_list)

### Pandas indexing

In [34]:
print(custom_ind._ts1)
print(custom_ind.ts1)

print(custom_ind.ts1.iloc[:, 0])
print(custom_ind.iloc[:, 0].ts1)

print(custom_ind.ts1.iloc[:, [0]])
print(custom_ind.iloc[:, [0]].ts1)

print(custom_ind.ts1.iloc[:2, :])
print(custom_ind.iloc[:2, :].ts1)

In [35]:
print(custom_ind.o1.iloc[:, 0])
%timeit big_custom_ind.o1.iloc[:, 0] # benchmark, 1 column

print(custom_ind.iloc[:, 0].o1) # performed on the object itself
%timeit big_custom_ind.iloc[:, 0] # slower since it forwards the operation to each dataframe

In [36]:
print(custom_ind.o1.iloc[:, np.arange(3)])
%timeit big_custom_ind.o1.iloc[:, np.arange(1000)] # 1000 columns

print(custom_ind.iloc[:, np.arange(3)].o1)
%timeit big_custom_ind.iloc[:, np.arange(1000)]

In [37]:
print(custom_ind.o1.loc[:, (1, 3, 'a')])
%timeit big_custom_ind.o1.loc[:, (1, 3, 0)] # 1 column

print(custom_ind.loc[:, (1, 3, 'a')].o1)
%timeit big_custom_ind.loc[:, (1, 3, 0)]

In [38]:
print(custom_ind.o1.loc[:, (1, 3)])
%timeit big_custom_ind.o1.loc[:, 1] # 1000 columns

print(custom_ind.loc[:, (1, 3)].o1)
%timeit big_custom_ind.loc[:, 1]

In [39]:
print(custom_ind.o1.xs(1, axis=1, level=0))
%timeit big_custom_ind.o1.xs(1, axis=1, level=0) # 1000 columns

print(custom_ind.xs(1, axis=1, level=0).o1)
%timeit big_custom_ind.xs(1, axis=1, level=0)

### Parameter indexing

In [40]:
# Indexing by parameter
print(custom_ind._p1_mapper)
print(custom_ind.p1_loc[2].o1)
print(custom_ind.p1_loc[1:2].o1)
print(custom_ind.p1_loc[[1, 1, 1]].o1)

In [41]:
%timeit big_custom_ind.p1_loc[1] # 1000 columns
%timeit big_custom_ind.p1_loc[np.full(10, 1)] # 10000 columns

In [42]:
print(custom_ind._tuple_mapper)
print(custom_ind.tuple_loc[(1, 3)].o1)
print(custom_ind.tuple_loc[(1, 3):(2, 4)].o1)

In [43]:
%timeit big_custom_ind.tuple_loc[(1, 3)]
%timeit big_custom_ind.tuple_loc[[(1, 3)] * 10]

### Comparison methods

In [44]:
print(custom_ind.o1 > 2)
%timeit big_custom_ind.o1.values > 2 # don't even try pandas

print(custom_ind.o1_above(2))
%timeit big_custom_ind.o1_above(2) # slower than numpy because of constructing dataframe

In [45]:
print(pd.concat((custom_ind.o1 > 2, custom_ind.o1 > 3), axis=1))
%timeit np.hstack((big_custom_ind.o1.values > 2, big_custom_ind.o1.values > 3))

print(custom_ind.o1_above([2, 3]))
%timeit big_custom_ind.o1_above([2, 3])

## TA-Lib

In [46]:
ts = pd.DataFrame({
    'a': [1, 2, 3, 4, np.nan],
    'b': [np.nan, 4, 3, 2, 1],
    'c': [1, 2, np.nan, 2, 1]
}, index=pd.DatetimeIndex([
    datetime(2018, 1, 1),
    datetime(2018, 1, 2),
    datetime(2018, 1, 3),
    datetime(2018, 1, 4),
    datetime(2018, 1, 5)
]))

In [47]:
SMA = vbt.talib('SMA')

print(SMA.run(close['a'], 2).real)
print(SMA.run(close, 2).real)
print(SMA.run(close, [2, 3]).real)

In [49]:
%timeit SMA.run(big_close)
%timeit SMA.run(big_close, np.arange(2, 10))
%timeit SMA.run(big_close, np.full(10, 2))
%timeit SMA.run(big_close, np.full(10, 2), run_unique=True)

In [50]:
comb = itertools.combinations(np.arange(2, 20), 2)
fast_windows, slow_windows = np.asarray(list(comb)).transpose()
print(fast_windows)
print(slow_windows)

In [51]:
%timeit SMA.run(big_close, fast_windows), SMA.run(big_close, slow_windows) # individual caching
%timeit SMA.run_combs(big_close, np.arange(2, 20)) # mutual caching

In [52]:
%timeit vbt.MA.run(big_close, fast_windows), vbt.MA.run(big_close, slow_windows) # the same using Numba
%timeit vbt.MA.run_combs(big_close, np.arange(2, 20))

In [53]:
sma1, sma2 = SMA.run_combs(close, [2, 3, 4])
print(sma1.real_crossed_above(sma2))
print(sma1.real_crossed_below(sma2))

In [54]:
dir(vbt.talib('BBANDS'))

## MA

In [55]:
print(close.rolling(2).mean())
print(close.ewm(span=3, min_periods=3).mean())
print(vbt.talib('SMA').run(close, timeperiod=2).real)
print(vbt.MA.run(close, [2, 3], ewm=[False, True]).ma)  # adjust=False

In [56]:
# One window
%timeit big_close.rolling(2).mean() # pandas
%timeit vbt.talib('SMA').run(big_close, timeperiod=2)
%timeit vbt.MA.run(big_close, 2, return_cache=True) # cache only
%timeit vbt.MA.run(big_close, 2) # with pre+postprocessing and still beats pandas

print(vbt.MA.run(big_close, 2).ma.shape)

In [57]:
# Multiple windows
%timeit pd.concat([big_close.rolling(i).mean() for i in np.arange(2, 10)])
%timeit vbt.talib('SMA').run(big_close, np.arange(2, 10))
%timeit vbt.MA.run(big_close, np.arange(2, 10))
%timeit vbt.MA.run(big_close, np.arange(2, 10), run_unique=True)
%timeit vbt.MA.run(big_close, np.arange(2, 10), return_cache=True) # cache only
cache = vbt.MA.run(big_close, np.arange(2, 10), return_cache=True)
%timeit vbt.MA.run(big_close, np.arange(2, 10), use_cache=cache) # using cache

print(vbt.MA.run(big_close, np.arange(2, 10)).ma.shape)

In [58]:
# One window repeated
%timeit pd.concat([big_close.rolling(i).mean() for i in np.full(10, 2)])
%timeit vbt.talib('SMA').run(big_close, np.full(10, 2))
%timeit vbt.MA.run(big_close, np.full(10, 2))
%timeit vbt.MA.run(big_close, np.full(10, 2), run_unique=True)  # slower for large inputs
%timeit vbt.MA.run(big_close, np.full(10, 2), return_cache=True)

print(vbt.MA.run(big_close, np.full(10, 2)).ma.shape)

In [59]:
%timeit pd.concat([big_close.iloc[:, :10].rolling(i).mean() for i in np.full(100, 2)])
%timeit vbt.talib('SMA').run(big_close.iloc[:, :10], np.full(100, 2))
%timeit vbt.MA.run(big_close.iloc[:, :10], np.full(100, 2))
%timeit vbt.MA.run(big_close.iloc[:, :10], np.full(100, 2), run_unique=True)  # faster for smaller inputs
%timeit vbt.MA.run(big_close.iloc[:, :10], np.full(100, 2), return_cache=True)

print(vbt.MA.run(big_close.iloc[:, :10], np.full(100, 2)).ma.shape)

In [60]:
ma = vbt.MA.run(close, [2, 3], ewm=[False, True])

print(ma.ma)

In [61]:
ma[(2, False, 'a')].plot().show_svg()

## MSTD

In [62]:
print(close.rolling(2).std(ddof=0))
print(close.ewm(span=3, min_periods=3).std(ddof=0))
print(vbt.talib('STDDEV').run(close, timeperiod=2).real)  
print(vbt.MSTD.run(close, [2, 3], ewm=[False, True]).mstd)  # adjust=False, ddof=0

In [63]:
# One window
%timeit big_close.rolling(2).std()
%timeit vbt.talib('STDDEV').run(big_close, timeperiod=2)
%timeit vbt.MSTD.run(big_close, 2)

print(vbt.MSTD.run(big_close, 2).mstd.shape)

In [64]:
# Multiple windows
%timeit pd.concat([big_close.rolling(i).std() for i in np.arange(2, 10)])
%timeit vbt.talib('STDDEV').run(big_close, timeperiod=np.arange(2, 10))
%timeit vbt.MSTD.run(big_close, np.arange(2, 10))

print(vbt.MSTD.run(big_close, np.arange(2, 10)).mstd.shape)

In [65]:
# One window repeated
%timeit vbt.talib('STDDEV').run(big_close, timeperiod=np.full(10, 2))
%timeit vbt.MSTD.run(big_close, window=np.full(10, 2))

print(vbt.MSTD.run(big_close, window=np.full(10, 2)).close.shape)

In [66]:
mstd = vbt.MSTD.run(close, [2, 3], [False, True])

print(mstd.mstd)

In [67]:
mstd[(2, False, 'a')].plot().show_svg()

## BBANDS

In [68]:
print(vbt.ta('BollingerBands').run(close['a'], window=2, window_dev=2).bollinger_hband)
print(vbt.ta('BollingerBands').run(close['a'], window=2, window_dev=2).bollinger_mavg)
print(vbt.ta('BollingerBands').run(close['a'], window=2, window_dev=2).bollinger_lband)

print(vbt.talib('BBANDS').run(close, timeperiod=2, nbdevup=2, nbdevdn=2).upperband)
print(vbt.talib('BBANDS').run(close, timeperiod=2, nbdevup=2, nbdevdn=2).middleband)
print(vbt.talib('BBANDS').run(close, timeperiod=2, nbdevup=2, nbdevdn=2).lowerband)

print(vbt.BBANDS.run(close, window=2, ewm=False, alpha=2).upper)
print(vbt.BBANDS.run(close, window=2, ewm=False, alpha=2).middle)
print(vbt.BBANDS.run(close, window=2, ewm=False, alpha=2).lower)

In [69]:
# One window
%timeit vbt.talib('BBANDS').run(big_close, timeperiod=2)
%timeit vbt.BBANDS.run(big_close, window=2)

print(vbt.BBANDS.run(big_close).close.shape)

In [70]:
# Multiple windows
%timeit vbt.talib('BBANDS').run(big_close, timeperiod=np.arange(2, 10))
%timeit vbt.BBANDS.run(big_close, window=np.arange(2, 10))

print(vbt.BBANDS.run(big_close, window=np.arange(2, 10)).close.shape)

In [71]:
# One window repeated
%timeit vbt.talib('BBANDS').run(big_close, timeperiod=np.full(10, 2))
%timeit vbt.BBANDS.run(big_close, window=np.full(10, 2))

print(vbt.BBANDS.run(big_close, window=np.full(10, 2)).close.shape)

In [72]:
bb = vbt.BBANDS.run(close, window=2, alpha=[1., 2.], ewm=False)

print(bb.middle)
print()
print(bb.upper)
print()
print(bb.lower)
print()
print(bb.percent_b)
print()
print(bb.bandwidth)

In [73]:
print(bb.close_below(bb.upper) & bb.close_above(bb.lower)) # price between bands

In [74]:
bb[(2, False, 1., 'a')].plot().show_svg()

## RSI

In [75]:
print(vbt.ta('RSIIndicator').run(close=close['a'], window=2).rsi)  # alpha=1/n
print(vbt.ta('RSIIndicator').run(close=close['b'], window=2).rsi)
print(vbt.ta('RSIIndicator').run(close=close['c'], window=2).rsi)
print(vbt.talib('RSI').run(close, timeperiod=2).real)
print(vbt.RSI.run(close, window=[2, 2], ewm=[True, False]).rsi)  # span=n

In [76]:
# One window
%timeit vbt.talib('RSI').run(big_close, timeperiod=2)
%timeit vbt.RSI.run(big_close, window=2)

print(vbt.RSI.run(big_close, window=2).rsi.shape)

In [77]:
# Multiple windows
%timeit vbt.talib('RSI').run(big_close, timeperiod=np.arange(2, 10))
%timeit vbt.RSI.run(big_close, window=np.arange(2, 10))

print(vbt.RSI.run(big_close, window=np.arange(2, 10)).rsi.shape)

In [78]:
# One window repeated
%timeit vbt.talib('RSI').run(big_close, timeperiod=np.full(10, 2))
%timeit vbt.RSI.run(big_close, window=np.full(10, 2))

print(vbt.RSI.run(big_close, window=np.full(10, 2)).rsi.shape)

In [79]:
rsi = vbt.RSI.run(close, window=[2, 3], ewm=[False, True])

print(rsi.rsi)

In [80]:
print(rsi.rsi_above(70))

In [81]:
rsi[(2, False, 'a')].plot().show_svg()

## STOCH

In [82]:
print(vbt.ta('StochasticOscillator').run(high=high['a'], low=low['a'], close=close['a'], window=2, smooth_window=3).stoch)
print(vbt.ta('StochasticOscillator').run(high=high['a'], low=low['a'], close=close['a'], window=2, smooth_window=3).stoch_signal)
print(vbt.talib('STOCHF').run(high, low, close, fastk_period=2, fastd_period=3).fastk)
print(vbt.talib('STOCHF').run(high, low, close, fastk_period=2, fastd_period=3).fastd)
print(vbt.STOCH.run(high, low, close, k_window=2, d_window=3).percent_k)
print(vbt.STOCH.run(high, low, close, k_window=2, d_window=3).percent_d)

In [83]:
# One window
%timeit vbt.talib('STOCHF').run(big_high, big_low, big_close, fastk_period=2)
%timeit vbt.STOCH.run(big_high, big_low, big_close, k_window=2)

print(vbt.STOCH.run(big_high, big_low, big_close, k_window=2).percent_d.shape)

In [84]:
# Multiple windows
%timeit vbt.talib('STOCHF').run(big_high, big_low, big_close, fastk_period=np.arange(2, 10))
%timeit vbt.STOCH.run(big_high, big_low, big_close, k_window=np.arange(2, 10))

print(vbt.STOCH.run(big_high, big_low, big_close, k_window=np.arange(2, 10)).percent_d.shape)

In [85]:
# One window repeated
%timeit vbt.talib('STOCHF').run(big_high, big_low, big_close, fastk_period=np.full(10, 2))
%timeit vbt.STOCH.run(big_high, big_low, big_close, k_window=np.full(10, 2))

print(vbt.STOCH.run(big_high, big_low, big_close, k_window=np.full(10, 2)).percent_d.shape)

In [86]:
stochastic = vbt.STOCH.run(high, low, close, k_window=[2, 4], d_window=2, d_ewm=[False, True])

print(stochastic.percent_k)
print(stochastic.percent_d)

In [87]:
stochastic[(2, 2, False, 'a')].plot().show_svg()

## MACD

In [88]:
print(vbt.ta('MACD').run(close['a'], window_fast=2, window_slow=3, window_sign=2).macd)
print(vbt.ta('MACD').run(close['a'], window_fast=2, window_slow=3, window_sign=2).macd_signal)
print(vbt.ta('MACD').run(close['a'], window_fast=2, window_slow=3, window_sign=2).macd_diff)

print(vbt.talib('MACD').run(close, fastperiod=2, slowperiod=3, signalperiod=2).macd)  # uses sma
print(vbt.talib('MACD').run(close, fastperiod=2, slowperiod=3, signalperiod=2).macdsignal)
print(vbt.talib('MACD').run(close, fastperiod=2, slowperiod=3, signalperiod=2).macdhist)

print(vbt.MACD.run(close, fast_window=2, slow_window=3, signal_window=2, macd_ewm=True, signal_ewm=True).macd)
print(vbt.MACD.run(close, fast_window=2, slow_window=3, signal_window=2, macd_ewm=True, signal_ewm=True).signal)
print(vbt.MACD.run(close, fast_window=2, slow_window=3, signal_window=2, macd_ewm=True, signal_ewm=True).hist)

In [89]:
# One window
%timeit vbt.talib('MACD').run(big_close, fastperiod=2)
%timeit vbt.MACD.run(big_close, fast_window=2)

print(vbt.MACD.run(big_close, fast_window=2).macd.shape)

In [90]:
# Multiple windows
%timeit vbt.talib('MACD').run(big_close, fastperiod=np.arange(2, 10))
%timeit vbt.MACD.run(big_close, fast_window=np.arange(2, 10))

print(vbt.MACD.run(big_close, fast_window=np.arange(2, 10)).macd.shape)

In [91]:
# One window repeated
%timeit vbt.talib('MACD').run(big_close, fastperiod=np.full(10, 2))
%timeit vbt.MACD.run(big_close, fast_window=np.full(10, 2))

print(vbt.MACD.run(big_close, fast_window=np.full(10, 2)).macd.shape)

In [92]:
macd = vbt.MACD.run(close, fast_window=2, slow_window=3, signal_window=[2, 3], macd_ewm=True, signal_ewm=True)

print(macd.macd)
print(macd.signal)
print(macd.hist)

In [93]:
macd[(2, 3, 2, True, True, 'a')].plot().show_svg()

## ATR

In [94]:
print(vbt.ta('AverageTrueRange').run(high['a'], low['a'], close['a'], window=2).average_true_range)
print(vbt.ta('AverageTrueRange').run(high['b'], low['b'], close['b'], window=2).average_true_range)
print(vbt.ta('AverageTrueRange').run(high['c'], low['c'], close['c'], window=2).average_true_range)
print(vbt.talib('ATR').run(high, low, close, timeperiod=2).real)
print(vbt.ATR.run(high, low, close, window=[2, 3], ewm=[False, True]).atr)

In [95]:
# One window
%timeit vbt.talib('ATR').run(big_high, big_low, big_close, timeperiod=2)
%timeit vbt.ATR.run(big_high, big_low, big_close, window=2)

print(vbt.ATR.run(big_high, big_low, big_close, window=2).atr.shape)

In [96]:
# Multiple windows
%timeit vbt.talib('ATR').run(big_high, big_low, big_close, timeperiod=np.arange(2, 10))
%timeit vbt.ATR.run(big_high, big_low, big_close, window=np.arange(2, 10)) # rolling min/max very expensive

print(vbt.ATR.run(big_high, big_low, big_close, window=np.arange(2, 10)).atr.shape)

In [97]:
# One window repeated
%timeit vbt.talib('ATR').run(big_high, big_low, big_close, timeperiod=np.full(10, 2))
%timeit vbt.ATR.run(big_high, big_low, big_close, window=np.full(10, 2))

print(vbt.ATR.run(big_high, big_low, big_close, window=np.full(10, 2)).atr.shape)

In [98]:
atr = vbt.ATR.run(high, low, close, window=[2, 3], ewm=[False, True])

print(atr.tr)
print(atr.atr)

In [99]:
atr[(2, False, 'a')].plot().show_svg()

## OBV

In [100]:
print(vbt.ta('OnBalanceVolumeIndicator').run(close['a'], volume['a']).on_balance_volume)
print(vbt.ta('OnBalanceVolumeIndicator').run(close['b'], volume['b']).on_balance_volume)
print(vbt.ta('OnBalanceVolumeIndicator').run(close['c'], volume['c']).on_balance_volume)
print(vbt.talib('OBV').run(close, volume).real)
print(vbt.OBV.run(close, volume).obv)

In [101]:
%timeit vbt.talib('OBV').run(big_close, big_volume)
%timeit vbt.OBV.run(big_close, big_volume)

print(vbt.OBV.run(big_close, big_volume).obv.shape)

In [102]:
obv = vbt.OBV.run(close, volume)

print(obv.obv)

In [103]:
print(obv.obv_above([0, 5]))

In [104]:
obv['a'].plot().show_svg()